In [32]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [33]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [34]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [35]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [36]:
data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
print(data_pk2_path)
print(media_pk2_path)

c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Data.pk2
c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Media.pk2


In [37]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [38]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [39]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [40]:
_media_file_stream = open(media_pk2_path, 'rb')

In [41]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [42]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [43]:
blowfish_checksum_decoded = "Joymax Pak File"

In [44]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [45]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [46]:
Root = read_blocks_at(256)

In [47]:
blocks_in_memory = {
  "": Root
}

In [48]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [49]:
server_dep = os.path.join('server_dep', 'silkroad', 'textdata')
server_dep

'server_dep\\silkroad\\textdata'

In [50]:
skill_text_data = dict()

## SkillData.txt

In [51]:
skill_data = os.path.join(server_dep, 'SkillData.txt')
skill_data

'server_dep\\silkroad\\textdata\\SkillData.txt'

In [52]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  


In [53]:
item_data_files = get_lines(skill_data)
item_data_files

['SkillData_100.txt',
 'SkillData_1000.txt',
 'SkillData_10000.txt',
 'SkillData_10100.txt',
 'SkillData_10200.txt',
 'SkillData_10300.txt',
 'SkillData_10400.txt',
 'SkillData_10500.txt',
 'SkillData_10600.txt',
 'SkillData_10700.txt',
 'SkillData_10800.txt',
 'SkillData_10900.txt',
 'SkillData_1100.txt',
 'SkillData_11000.txt',
 'SkillData_11100.txt',
 'SkillData_11200.txt',
 'SkillData_11300.txt',
 'SkillData_11400.txt',
 'SkillData_11500.txt',
 'SkillData_11600.txt',
 'SkillData_11700.txt',
 'SkillData_11800.txt',
 'SkillData_11900.txt',
 'SkillData_1200.txt',
 'SkillData_12000.txt',
 'SkillData_12100.txt',
 'SkillData_12200.txt',
 'SkillData_12300.txt',
 'SkillData_12400.txt',
 'SkillData_12500.txt',
 'SkillData_12600.txt',
 'SkillData_12700.txt',
 'SkillData_12800.txt',
 'SkillData_12900.txt',
 'SkillData_1300.txt',
 'SkillData_13000.txt',
 'SkillData_13100.txt',
 'SkillData_13200.txt',
 'SkillData_13300.txt',
 'SkillData_13400.txt',
 'SkillData_13500.txt',
 'SkillData_13600.txt'

In [54]:
for xxx in item_data_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 11:
      continue
    
    Service = values[0]
    ID = values[1]
    GroupID = values[2]
    Basic_Code = values[3]
    Basic_Level = values[7]
    Basic_Activity = values[8]
    Basic_ChainCode = values[9]

    Action_PreparingTime = values[11]
    Action_CastingTime = values[12]
    Action_ActionDuration = values[13]
    Action_ResuseDelay = values[14]
    Action_Overlap = values[18]
    Action_AutoAttackType = values[19]

    Action_Range = values[21]
    Target_Required = values[22]
    TargetType_Animal = values[23]

    TargetGroup_Self = values[26]
    TargetGroup_Ally = values[27]
    TargetGroup_Party = values[28]
    TargetGroup_Enemy_M = values[29]
    TargetGroup_Enemy_P = values[30]

    TargetEtc_SelectDeadBody = values[33]
    ReqCommon_Mastery1 = values[34]
    ReqCommon_Mastery2 = values[35]

    ReqCommon_MasteryLevel1 = values[36]
    ReqCommon_MasteryLevel2 = values[37]

    ReqCast_Weapon1 = values[50]
    ReqCast_Weapon2 = values[51]

    Consume_MP = values[53]

    UI_IconFile = values[61]
    UI_SkillName = values[62]

    PARAM_COUNT = 50
    Params = []
    for i in range(PARAM_COUNT):
      Params.append(values[63 + i])

    skill_text_data[ID] = {
      'Service': Service,
      'ID': ID,
      'GroupID': GroupID,
      'Basic_Code': Basic_Code,
      'Basic_Level': Basic_Level,
      'Basic_Activity': Basic_Activity,
      'Basic_ChainCode': Basic_ChainCode,
      'Action_PreparingTime': Action_PreparingTime,
      'Action_CastingTime': Action_CastingTime,
      'Action_ActionDuration': Action_ActionDuration,
      'Action_ResuseDelay': Action_ResuseDelay,
      'Action_Overlap': Action_Overlap,
      'Action_AutoAttackType': Action_AutoAttackType,
      'Action_Range': Action_Range,
      'Target_Required': Target_Required,
      'TargetType_Animal': TargetType_Animal,
      'TargetGroup_Self': TargetGroup_Self,
      'TargetGroup_Ally': TargetGroup_Ally,
      'TargetGroup_Party': TargetGroup_Party,
      'TargetGroup_Enemy_M': TargetGroup_Enemy_M,
      'TargetGroup_Enemy_P': TargetGroup_Enemy_P,
      'TargetEtc_SelectDeadBody': TargetEtc_SelectDeadBody,
      'ReqCommon_Mastery1': ReqCommon_Mastery1,
      'ReqCommon_Mastery2': ReqCommon_Mastery2,
      'ReqCommon_MasteryLevel1': ReqCommon_MasteryLevel1,
      'ReqCommon_MasteryLevel2': ReqCommon_MasteryLevel2,
      'ReqCast_Weapon1': ReqCast_Weapon1,
      'ReqCast_Weapon2': ReqCast_Weapon2,
      'Consume_MP': Consume_MP,
      'UI_IconFile': UI_IconFile,
      'UI_SkillName': UI_SkillName,
      'Params': Params
    }

In [55]:
skill_text_data

{'1': {'Service': '1',
  'ID': '1',
  'GroupID': '172',
  'Basic_Code': 'SKILL_PUNCH_01',
  'Basic_Level': '1',
  'Basic_Activity': '2',
  'Basic_ChainCode': '0',
  'Action_PreparingTime': '0',
  'Action_CastingTime': '0',
  'Action_ActionDuration': '1500',
  'Action_ResuseDelay': '1500',
  'Action_Overlap': '0',
  'Action_AutoAttackType': '1',
  'Action_Range': '0',
  'Target_Required': '1',
  'TargetType_Animal': '1',
  'TargetGroup_Self': '0',
  'TargetGroup_Ally': '0',
  'TargetGroup_Party': '0',
  'TargetGroup_Enemy_M': '1',
  'TargetGroup_Enemy_P': '1',
  'TargetEtc_SelectDeadBody': '0',
  'ReqCommon_Mastery1': '0',
  'ReqCommon_Mastery2': '0',
  'ReqCommon_MasteryLevel1': '0',
  'ReqCommon_MasteryLevel2': '0',
  'ReqCast_Weapon1': '1',
  'ReqCast_Weapon2': '255',
  'Consume_MP': '0',
  'UI_IconFile': 'xxx',
  'UI_SkillName': 'SN_SKILL_PUNCH',
  'Params': ['xxx',
   'xxx',
   'xxx',
   '0',
   '0',
   '0',
   '6386804',
   '5',
   '150',
   '0',
   '0',
   '150',
   '0',
   '0',


In [56]:
df = pd.DataFrame.from_dict(skill_text_data, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'Key'}, inplace=True)

In [57]:
csv_file_path = 'skill_data.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8')

In [58]:
skill_mastery_text_data = dict()

## SkillMasteryData.txt

In [59]:
skill_mastery = os.path.join(server_dep, 'SkillMasteryData.txt')
skill_mastery

'server_dep\\silkroad\\textdata\\SkillMasteryData.txt'

In [60]:
csv_file_path = 'data_text.csv'

df = pd.read_csv(csv_file_path, encoding='utf-8', index_col=0)

In [61]:
df

,Service,Data
Key,,
UIC_STT_FONTNAME,1,Tahoma
UIIS_STT_MSG_INSTALL_START,1,"Cài đặt ""Con Đường Tơ Lụa"""
UIIS_STT_MSG_INSTALL_FOLDER,1,"Cài đặt ""Con Đường Tơ Lụa"" vào thư mục sau"
UIIS_STT_MSG_INSTALL_END,1,Con Đường Tơ Lụa đã được cài đặt thành công
UIIS_STT_INSTALL_FOLDER,1,Thư mục cài đặt
...,...,...
SN_TALK_QNO_N_KT_082_02,1,Giết 308 Lam Băng Binh và 330 Khôi Băng Binh.
SN_TALK_QNO_N_KT_082_03,1,Chúng quá mạnh mẽ so với bạn?
SN_TALK_QNO_N_KT_082_04,1,Giết 308 Lam Băng Binh và 330 Khôi Băng Binh.


In [62]:
df.loc['UIC_STT_FONTNAME']['Data']

'Tahoma'

In [63]:
lines = get_lines(skill_mastery)
for idx, line in enumerate(lines):
    try:
      values = line.split('\t')
      if len(values) < 11:
        continue

      ID = values[0]
      NameCode = values[3]
      Name = df.loc[NameCode]['Data']

      skill_mastery_text_data[ID] = {
        'ID': ID,
        'Name': Name,
        'NameCode': NameCode
      }
    except Exception as e:
      continue

In [64]:
skill_mastery_text_data

{'257': {'ID': '257', 'Name': 'Phi Thiên', 'NameCode': 'UIIT_STT_MASTERY_VI'},
 '258': {'ID': '258', 'Name': 'Hắc Sát', 'NameCode': 'UIIT_STT_MASTERY_HEUK'},
 '259': {'ID': '259', 'Name': 'Phá Thiên', 'NameCode': 'UIIT_STT_MASTERY_PA'},
 '277': {'ID': '277', 'Name': 'Phép', 'NameCode': 'UIIT_CTL_FORCE_SKILL'},
 '276': {'ID': '276', 'Name': 'Nội Công', 'NameCode': 'UIIT_STT_MASTERY_GI'},
 '513': {'ID': '513', 'Name': 'Chiến Binh', 'NameCode': 'UIIT_STT_WARRIOR'},
 '515': {'ID': '515', 'Name': 'Đạo Chích', 'NameCode': 'UIIT_STT_ROG'},
 '514': {'ID': '514', 'Name': 'Phù Thủy', 'NameCode': 'UIIT_STT_WIZARD'},
 '516': {'ID': '516', 'Name': 'Thuật Sĩ', 'NameCode': 'UIIT_STT_WARLOCK'},
 '517': {'ID': '517', 'Name': 'Nhạc Công', 'NameCode': 'UIIT_STT_BARD'},
 '518': {'ID': '518', 'Name': 'Mục Sư', 'NameCode': 'UIIT_STT_CLERIC'},
 '1000': {'ID': '1000',
  'Name': 'Kỹ năng',
  'NameCode': 'UIIT_STT_CONFLICT_JOB_SKILL'}}

In [65]:
df = pd.DataFrame.from_dict(skill_mastery_text_data, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'Key'}, inplace=True)

In [66]:
csv_file_path = 'mastery_data.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8')